In [44]:
import ast
import math
import os, gc
import random
from tqdm import tqdm
from pathlib import Path

import numpy as np
import pandas as pd

import cv2
from PIL import Image
from skimage.measure import label, regionprops

import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors as colors
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap, Normalize
from mpl_toolkits.axes_grid1 import make_axes_locatable


import torch
import torch.backends.cudnn as cudnn
import torch.utils.data as data
import torch.nn.functional as F

from dataset import DeployDataset
from network.textnet import TextNet
from cfglib.config import config as cfg, update_config, print_config
from cfglib.option import BaseOptions
from util.augmentation import BaseTransform
from util.visualize import visualize_detection, visualize_gt
from util.misc import to_device, mkdirs,rescale_result


import multiprocessing
multiprocessing.set_start_method("spawn", force=True)


jet_cmap = LinearSegmentedColormap.from_list('jet', ['black', 'blue', 'green', 'yellow', 'red', 'white'], N = 256)

In [35]:
def osmkdir(out_dir):
    import shutil
    if os.path.exists(out_dir):
        shutil.rmtree(out_dir)
    os.makedirs(out_dir)

In [36]:
def heatmap(im_gray):
    cmap = plt.get_cmap('jet')
    rgba_img = cmap(255 - im_gray)
    Hmap = np.delete(rgba_img, 3, 2)
    return Hmap

In [43]:
def format_bbox(boxes, mask_shapes = None, img_shape = None, mask_type = 'gt'):
    boxes_clean = []
    for box_coords in boxes:
        if mask_type == 'mask2box':
            mult = img_shape[0]/mask_shapes[0], img_shape[1]/mask_shapes[1]
            boxes_clean.append([
                [box_coords.bbox[1]*mult[1], box_coords.bbox[0]*mult[0]],
                [box_coords.bbox[3]*mult[1], box_coords.bbox[0]*mult[0]],
                [box_coords.bbox[3]*mult[1], box_coords.bbox[2]*mult[0]],
                [box_coords.bbox[1]*mult[1], box_coords.bbox[2]*mult[0]]
            ])
    return np.asarray(boxes_clean)

def create_mask(boxes, image_shape):
    mask = np.zeros(image_shape[:2], dtype = 'uint8')
    for box_coords in boxes:
        top_left = [math.ceil(val) for val in box_coords[0]]
        bottom_right = [math.ceil(val) for val in box_coords[2]]
        mask[top_left[1]:bottom_right[1], top_left[0]: bottom_right[0]] = 1
    return mask

def mask2box(mask, img_shape, bbox_thresh = 0.6):
    mask_shapes = mask.shape
    mask = mask > bbox_thresh
    lbl_0 = label(mask)
    props = regionprops(lbl_0)
    return format_bbox(props, mask_shapes, img_shape, mask_type = 'mask2box')

In [2]:
args = {
    "img_root": "/home/lkhagvadorj/Temuujin/gcv/tmp/",
    "net": "resnet18",
    "scale": 4,
    "exp_name": 'TD500',
    "checkepoch": 1135,
    "test_size": (640, 960),
    "dis_threshold": 0.35,
    "cls_threshold": 0.9,
    "gpu": "0",
    "resume": False,
    "save_dir": './model/',
}
update_config(cfg, args)
print_config(cfg)

==========Options============
means: (0.485, 0.456, 0.406)
stds: (0.229, 0.224, 0.225)
gpu: 0
exp_name: TD500
num_workers: 24
batch_size: 12
max_epoch: 200
start_epoch: 0
lr: 0.0001
cuda: True
output_dir: output
input_size: 640
max_annotation: 64
adj_num: 4
num_points: 20
use_hard: True
load_memory: False
scale: 4
grad_clip: 25
dis_threshold: 0.35
cls_threshold: 0.9
approx_factor: 0.004
img_root: /home/lkhagvadorj/Temuujin/gcv/tmp/
net: resnet18
checkepoch: 1135
test_size: (640, 960)
resume: False
save_dir: ./model/
device: cuda
=============End=============


In [3]:
testset = DeployDataset(
    image_root = cfg.img_root,
    transform = BaseTransform(size=cfg.test_size, mean=cfg.means, std=cfg.stds)
)

if cfg.cuda:
    cudnn.benchmark = True

# Data
test_loader = data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=cfg.num_workers)

# Model
model = TextNet(is_training=False, backbone=cfg.net)
model_path = os.path.join(cfg.save_dir, cfg.exp_name,
                            'TextBPN_{}_{}.pth'.format(model.backbone_name, cfg.checkepoch))

model.load_model(model_path)
model = model.to(cfg.device)  # copy to cuda

Loading from ./model/TD500/TextBPN_resnet18_1135.pth


In [6]:
model.eval()
with torch.no_grad():
    print('Start testing TextBPN++.')
    output_dir = os.path.join(cfg.output_dir, cfg.exp_name)
    osmkdir(output_dir)
    
    for i, (image, meta) in enumerate(test_loader):
        input_dict = dict()
        idx = 0  # test mode can only run with batch_size == 1
        H, W = meta['Height'][idx].item(), meta['Width'][idx].item()
        print(meta['image_id'], (H, W))

        input_dict['img'] = to_device(image)

        # get detection result
        output_dict = model(input_dict)
        
        img_show = image[idx].permute(1, 2, 0).cpu().numpy()
        img_show = ((img_show * cfg.stds + cfg.means) * 255).astype(np.uint8)

Start testing TextBPN++.
['gcv_0236.jpg'] (2334, 1634)


In [19]:
img_show = image[idx].permute(1, 2, 0).cpu().numpy()
img_show = ((img_show * cfg.stds + cfg.means) * 255).astype(np.uint8)

image_show = img_show.copy()
image_show = np.ascontiguousarray(image_show[:, :, ::-1])

cls_preds = F.interpolate(output_dict["fy_preds"], scale_factor=cfg.scale, mode='bilinear')
cls_preds = cls_preds[0].data.cpu().numpy()

cls_pred = heatmap(np.array(cls_preds[0] * 255, dtype=np.uint8))
dis_pred = heatmap(np.array(cls_preds[1] * 255, dtype=np.uint8))

heat_map = np.concatenate([cls_pred*255, dis_pred*255], axis=1)
heat_map = cv2.resize(heat_map, (320 * 2, 320))

In [75]:
import matplotlib
matplotlib.use('module://ipykernel.pylab.backend_inline')

fig, axs = plt.subplots(2, 2, num = 1, clear = True, figsize = (10, 10))

vmin = cls_preds[0].min()
vmax = cls_preds[0].max()
centered_log_norm = colors.SymLogNorm(linthresh = 0.03, linscale = 0.03, 
                                      vmin = vmin, vmax = vmax, base = 10)

pred_heatmap = axs[0][0].imshow(cls_preds[0], norm = centered_log_norm, cmap = jet_cmap)
divider = make_axes_locatable(axs[0][0])
cax = divider.append_axes('right', size = '5%', pad = 0.05)
fig.colorbar(pred_heatmap, cax = cax, orientation = 'vertical')

axs[0][0].set_title("Pred Mask")
axs[0][0].axis('off')

vmin = cls_preds[1].min()
vmax = cls_preds[1].max()
centered_log_norm = colors.SymLogNorm(linthresh = 0.03, linscale = 0.03, 
                                      vmin = vmin, vmax = vmax, base = 10)

pred_heatmap = axs[0][1].imshow(cls_preds[0], norm = centered_log_norm, cmap = jet_cmap)
divider = make_axes_locatable(axs[0][1])
cax = divider.append_axes('right', size = '5%', pad = 0.05)
fig.colorbar(pred_heatmap, cax = cax, orientation = 'vertical')

axs[0][1].set_title("Distance Mask")
axs[0][1].axis('off')


img = np.asarray(Image.open(meta['image_path'][0]))
pred_bbox = [mask2box(cls_preds[0], (H, W), bbox_thresh = 0.8), mask2box(cls_preds[1], (H, W), bbox_thresh = 0.8)]
for idx, pred_bboxes in enumerate(pred_bbox):
    axs[1][idx].imshow(img)
    axs[1][idx].axis('off')
    
    axs[1][idx].set_title("Bbox thresh: 0.8")

    for ((x1, y1), (x2, y2), (x3, y3), (x4, y4)) in pred_bboxes:
        width = x3 - x1
        height = y3 - y1
        rect = patches.Rectangle((x1, y1), width, height, linewidth = 1, edgecolor = 'red', facecolor = 'none')
        axs[1][idx].add_patch(rect)

        axs[1][idx].plot(x1, y1, marker = 'o', markersize = 2, color = 'r')
        axs[1][idx].plot(x2, y2, marker = 'o', markersize = 2, color = 'g')
        axs[1][idx].plot(x3, y3, marker = 'o', markersize = 2, color = 'b')
        axs[1][idx].plot(x4, y4, marker = 'o', markersize = 2, color = 'pink')

fig.tight_layout()
fig.savefig('tmp_2.png', dpi = 150)

del fig, axs
gc.collect()

92465